In [3]:
import data_provider
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

ModuleNotFoundError: No module named 'torch'

In [ ]:
class Net(nn.Module):

    def __init__(self, input_size, layers_size: list, output_size, activiation_fn = 'relu', apply_dropout=False):
        super(Net, self).__init__()
        self.hidden_layers = []

        last_layer_size = input_size
        for layer_size in layers_size:
            self.layers += [nn.Linear(last_layer_size, layer_size)]
            last_layer_size = layer_size
        self.output = nn.Linear(last_layer_size, output_size)

        self.activation_fn = activiation_fn

        self.dropout = None
        if apply_dropout:
            self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        activation_fn = None
        if self.activation_fn == 'sigmoid':
            activation_fn = F.sigmoid
        elif self.activation_fn == 'tanh':
            activation_fn = torch.tanh
        elif self.activation_fn == 'relu':
            activation_fn = F.relu

        for layer in self.hidden_layers:
            x = activation_fn(layer(x))

        if self.dropout is not None :
            x = self.dropout(x)
        x = self.output(x)
        return F.log_softmax(x, dim=-1)

    def train_model(self, x, y, epochs=100, learning_rate=0.1):
        optimizer = optim.SGD(self.parameters(), lr=learning_rate)
        loss_fn = nn.NLLLoss()

        for epoch in range(epochs):
            optimizer.zero_grad()

            self.train()
            pred = self(x)
            loss = loss_fn(pred, y)
            loss.backward()

            optimizer.step()

In [ ]:
for dataset in data_provider.data():
    features_train = dataset['train'][0]
    labels_train = dataset['train'][1]

    features_test = dataset['test'][0]
    labels_test = dataset['test'][1]

    model = Net(input_size=len(features_train[0]), layers_size=[100], output_size=1)

    model.train_model()